In [16]:
import sys,pysam

def pileup_bam(input_line,bam_file):
    input_split = input_line.split('\t')
    input_chr = input_split[0]
    input_pos = long(input_split[1])
    input_ref = input_split[3]
    input_alt = input_split[4][0:1]
    i=0
    nref=0
    nalt=0
    for pileupcolumn in bam_file.pileup(input_chr,input_pos-1,input_pos,min_mapping_quality=0,min_base_quality=10):
        #if pileupcolumn.pos == input_pos-1:
            #print pileupcolumn.get_query_qualities()
        for pileupread in pileupcolumn.pileups:
            if pileupcolumn.pos == input_pos-1:
                if pileupread.query_position == None:
                    continue
                else:
                    if pileupread.alignment.query_sequence[pileupread.query_position] == input_ref:
                        nref+=1
                    elif pileupread.alignment.query_sequence[pileupread.query_position] == input_alt:
                        nalt+=1
    return [str(nalt),str(nref),str(round(float(nalt)/float(nref+nalt),2))]

def snp_input_rel_pos_fc(cigar_list, abs_pos, rel_pos, input_pos):
    target = ''
    remain_distance = input_pos - abs_pos
    for cigar in cigar_list:
        if cigar[0] == 0:
            if (abs_pos + cigar[1]) < input_pos:
                abs_pos += cigar[1]
                rel_pos += cigar[1]
                remain_distance -= cigar[1]
            elif (abs_pos + cigar[1]) == input_pos:
                abs_pos += cigar[1]
                rel_pos += cigar[1]
                remain_distance -= cigar[1]
                break
            else:
                rel_pos += remain_distance
                abs_pos += remain_distance
                remain_distance = 0
                break
        elif cigar[0] == 1:                        
            if abs_pos < input_pos:
                rel_pos += cigar[1] # no change of abs_pos and remain_distance  
            else:
                print 'unexpected result of snp_input_rel_pos_fc_cigar[0]==1'
                print input_pos
                break                           
        elif cigar[0] == 2:                                           
            if (abs_pos + cigar[1]) < input_pos:
                abs_pos += cigar[1] # no change of rel_pos 
                remain_distance -= cigar[1]
            else:
                abs_pos += remain_distance 
                rel_pos = 'Null' # there is no real rel.position of input_pos
                remain_distance = 0
                break                                
        elif cigar[0] == 3:
            if (abs_pos + cigar[1]) < input_pos:
                abs_pos += cigar[1]
                rel_pos += cigar[1]
                remain_distance -= cigar[1]                         
            else:
                rel_pos += remain_distance
                abs_pos += remain_distance
                remain_distance = 0
                break
        elif cigar[0] == 4 or cigar[0] == 5:
            rel_pos += remain_distance
            abs_pos += remain_distance
            remain_distance = 0                    
            break
        else:
            print 'cigar[0]>5, unexpected'
            print read
            break                
    return [abs_pos, rel_pos]
    
def snp_bam(input_line,tbam_file,nbam_file):
    input_split = input_line.split('\t')
    input_chr = input_split[0]
    input_pos = long(input_split[1])
    input_ref = input_split[3]
    input_alt = input_split[4][0:1]
    
    variant_list = []
    nonvariant_list = []
    normal_list = []
    
    t_var=0;t_ref=0;t_other=0;t_dup=0;n_var=0;n_ref=0;n_other=0;n_dup=0
    
    whether_ins = 0
    insertion_length = 0
    region_length = 0
    
    for read in nbam_file.fetch(input_chr,input_pos-1,input_pos):
        if read.cigartuples == None:
            continue
        else:
            whether_ins = 0
            insertion_length = 0
            region_length = 0
    
            ins_abs_pos = read.reference_start # include clip
            ins_rel_pos = read.query_alignment_start # consider left softclip, hardclip (ex> cigar: 10S141M => 10)
            
            input_pos_list = [] #[absolute position of input_pos, relative position of input_pos]
            cigar_list = read.cigartuples
            
            input_pos_list = snp_input_rel_pos_fc(cigar_list, read.reference_start, read.query_alignment_start, input_pos)
            input_abs_pos = input_pos_list[0]
            input_rel_pos = input_pos_list[1]
            
            if input_rel_pos == 'Null':
                n_other += 1
                continue
            
            if read.query_sequence[input_rel_pos - 1:input_rel_pos] == input_alt:
                n_var += 1
            else:
                n_ref += 1

                for cigar in cigar_list:
                    if cigar[0] == 1:
                        whether_ins += 1
                    else:
                        'blank'

                if whether_ins >1:
                    print 'whether_ins>1n'
                    print input_line
                    print read
                    sys.exit(1)
                elif whether_ins == 0:
                    'blank'
                else:
                    for cigar in cigar_list:
                        if cigar[0] == 0 or cigar[0] == 3:
                            ins_abs_pos += cigar[1]
                            ins_rel_pos += cigar[1]
                        elif cigar[0] == 1:
                            insertion_length = cigar[1]
                            break
                        elif cigar[0] == 2:
                            ins_abs_pos += cigar[1] # no change of rel_pos 
                        else:
                            print 'cigar[0]>3, unexpected in finding insertion1'
                            print input_line
                            print read
                            sys.exit(1)

                    region_length = ins_abs_pos - input_abs_pos
                    if region_length < 0:
                        temp_sequence = read.query_sequence[ins_rel_pos:ins_rel_pos-region_length] #region_length
                    else:
                        temp_sequence = read.query_sequence[ins_rel_pos + insertion_length - region_length - 1:ins_rel_pos + insertion_length] #region_length + 1 

                    temp_normal = [region_length,temp_sequence]
                    normal_list.append(temp_normal)  
      
    for read in tbam_file.fetch(input_chr,input_pos-1,input_pos):  # find nonvariant read in tumor_bam  
        if read.cigartuples == None:
            continue
        else:    
            whether_ins = 0
            insertion_length = 0
            region_length = 0
            
            ins_abs_pos = read.reference_start # include clip
            ins_rel_pos = read.query_alignment_start # consider left softclip, hardclip (ex> cigar: 10S141M => 10)
            
            input_pos_list = [] #[absolute position of input_pos, relative position of input_pos]
            cigar_list = read.cigartuples
            
            input_pos_list = snp_input_rel_pos_fc(cigar_list, read.reference_start, read.query_alignment_start, input_pos)
            input_abs_pos = input_pos_list[0]
            input_rel_pos = input_pos_list[1]
            
            if input_rel_pos == 'Null':
                t_other += 1
                continue
                
            if read.query_sequence[input_rel_pos - 1:input_rel_pos] == input_alt:
                t_var += 1
            else:
                t_ref += 1
                for cigar in cigar_list:
                    if cigar[0] == 1:
                        whether_ins += 1
                    else:
                        'blank'

                if whether_ins >1:
                    print 'whether_ins>1t'
                    print input_line
                    print read
                    sys.exit(1)
                elif whether_ins == 0:
                    'blank'
                else:                    
                    for cigar in cigar_list:
                        if cigar[0] == 0 or cigar[0] == 3:
                            ins_abs_pos += cigar[1]
                            ins_rel_pos += cigar[1]
                        elif cigar[0] == 1:
                            insertion_length = cigar[1]
                            break
                        elif cigar[0] == 2:
                            ins_abs_pos += cigar[1] # no change of rel_pos 
                        else:
                            print 'cigar[0]>3, unexpected in finding insertion2'
                            print input_line
                            print read
                            sys.exit(1)

                    region_length = ins_abs_pos - input_abs_pos
                    if region_length < 0:
                        temp_sequence = read.query_sequence[ins_rel_pos:ins_rel_pos-region_length] #region_length
                    else:
                        temp_sequence = read.query_sequence[ins_rel_pos + insertion_length - region_length - 1:ins_rel_pos + insertion_length] #region_length + 1 

                    temp_nonvariant = [region_length,temp_sequence]
                    nonvariant_list.append(temp_nonvariant)
                    
                    
    ni_list = []

    for read in tbam_file.fetch(input_chr,input_pos-1,input_pos):   # compare variant sequence vs (nonvariant & normal sequence)
        ni = 0
        ti = 0
        
        if read.cigartuples == None:
            continue
        else:
            cigar_list = read.cigartuples            
            input_pos_list = snp_input_rel_pos_fc(cigar_list, read.reference_start, read.query_alignment_start, input_pos)
            input_abs_pos = input_pos_list[0]
            input_rel_pos = input_pos_list[1]
            
            if read.query_sequence[input_rel_pos - 1:input_rel_pos] != input_alt:
                'blank'
            else:
                for normal in normal_list:
                    if ni in ni_list:
                        'blank'
                    else:                    
                        if normal[0] < 0:
                            var_sequence = read.query_sequence[input_rel_pos + normal[0] : input_rel_pos]                    
                        else:
                            var_sequence = read.query_sequence[input_rel_pos - 1 : input_rel_pos + normal[0]]

                        if var_sequence == normal[1]:
                            n_dup += 1
                            ni_list.append(ni)
                    ni += 1

                for nonvariant in nonvariant_list:
                    if nonvariant[0] < 0:
                        var_sequence = read.query_sequence[input_rel_pos + nonvariant[0] : input_rel_pos]                    
                    else:
                        var_sequence = read.query_sequence[input_rel_pos - 1 : input_rel_pos + nonvariant[0]]

                    if var_sequence == normal[1]:
                        t_dup +=1
                        break
    #print normal_list, len(normal_list)
    #print t_var,t_ref,t_other,t_dup,n_var,n_ref,n_other,n_dup
        
    return [t_var,t_ref,t_other,t_dup,n_var,n_ref,n_other,n_dup]                        
                            
    
#input_fn = sys.argv[1]
#input_file = file(input_fn)
#output_file = file(input_fn.replace('.vcf','_myfiltered.vcf'),'w')
input_file = file('S1-2Gy-1.combination.Mutect_Strelka_union.vcf')
#output_file = file('S1-2Gy-1.combination.Mutect_Strelka_union_test.vcf','w')

#t_bam_fn = sys.argv[2]
#n_bam_fn = sys.argv[3]
#t_bam = pysam.AlignmentFile(t_bam_fn,'rb')
#n_bam = pysam.AlignmentFile(n_bam_fn,'rb')
t_bam = pysam.AlignmentFile('/home/users/jhyouk/06_mm10_SNUH_radiation/03_bam/S1-2Gy-1.s.md.ir.br.rg.bam','rb')
n_bam = pysam.AlignmentFile('/home/users/jhyouk/06_mm10_SNUH_radiation/03_bam/N1spleen.s.md.ir.br.rg.bam','rb')
ref_fasta = pysam.FastaFile("/home/users/jhyouk/99_reference/mouse/mm10/GRCm38.fa")

input_line = input_file.readline().strip()
while input_line[0:1] == '#':
    #output_file.write(input_line + '\t' + 't_var;t_ref;t_other;t_dup;n_var;n_ref;n_other;n_dup;Vaf;indel;repeat_unit;reference_repeat_count' + '\n')
    #output_file.write(input_line + '\t' + 'initial_vaf' + '\n')
    input_line = input_file.readline().strip()

prev_chr = '0'

while input_line:
    input_chr = input_line.split('\t')[0]
    if input_chr != prev_chr:
        print input_chr
        prev_chr = input_chr
        
    if input_line.split('\t')[1] != '36980322':
        input_line = input_file.readline().strip()
        continue       

    info = ''

    #initial_vaf = pileup_bam(input_line,t_bam) # Calculate initial_vaf from bam file
    #info = initial_vaf[2]
    
    var_list = snp_bam(input_line,t_bam,n_bam)
    print var_list 
    break
    #output_file.write(input_line + '\t' + info + '\n')
    input_line = input_file.readline().strip()
    
print 'The End'

1
2
3
4
5
6
7
8
9
10
11
12
GACAGACAGACAGACAGACAGACAGACAGACAGACAGACAGACAGAC
[18, 10, 0, 0, 0, 50, 0, 27]
The End
